In [ ]:
import numpy as np
import pandas as pd

from airfoil._airfoil import Airfoil, Hole, Decomposer

from airfoil.cnc._serial import CNC
from airfoil.util.array_helpers import blur1d, map_to_range, remove_sequential_duplicates
from airfoil.util.linestring_helpers import deflection_angle_padded
from airfoil.util.path_planning import (
    project_line_to_plane
)
from airfoil.util.pyvista_helpers import create_ruled_surface

import pyvista as pv
pv.set_jupyter_backend('trame')


In [ ]:
holes = [
    Hole(5,np.array([50,2])),
    Hole(5,np.array([100,2])),
]
afa = Airfoil.from_naca4(
    max_camber=0,
    max_camber_position=0,
    max_thickness=0.12,
    chord_length=180,
).with_holes(holes)
afb = Airfoil.from_naca4(
    max_camber=0,
    max_camber_position=0,
    max_thickness=0.10,
    chord_length=150,
).with_translation(np.array([30,3])).with_holes(holes)
afa, afb = afb,afa
ax,counts=afa.plot()
afb.plot(ax=ax)

In [ ]:
decomposer = Decomposer(buffer=0.5)

In [ ]:
afa_paths = decomposer.decompose(afa)
afb_paths = decomposer.decompose(afb)

In [ ]:

afa_path = remove_sequential_duplicates(np.concat(afa_paths))
afb_path = remove_sequential_duplicates(np.concat(afb_paths))

all_points = np.concat([afa_path,afb_path])
bounds_min = all_points.min(axis=0)

afa_path-=bounds_min
afb_path-=bounds_min


foam_width    = 200
plane_spacing = 222

left_foam   = -foam_width    / 2
left_plane  = -plane_spacing / 2
right_foam  =  foam_width    / 2
right_plane =  plane_spacing / 2

afa_path_3d = np.concat([
    afa_path,
    np.ones(len(afa_path)).reshape(-1,1) * left_foam
], axis=-1)

afb_path_3d = np.concat([
    afb_path,
    np.ones(len(afb_path)).reshape(-1,1) * right_foam
], axis=-1)

afa_projected = []
afb_projected = []
for a, b in zip(afa_path_3d, afb_path_3d):
    afa_projected.append(project_line_to_plane(a, b, "yx", left_plane))
    afb_projected.append(project_line_to_plane(a, b, "yx", right_plane))
afa_projected = np.array(afa_projected)
afb_projected = np.array(afb_projected)

In [ ]:
speed = map_to_range(
    blur1d(np.max([
        deflection_angle_padded(afa_path),
        deflection_angle_padded(afb_path),
    ],axis=0)),
    300,
    200
)
pd.Series(speed).plot(xlabel="point",ylabel="mm/s", title="Cut Speed", figsize=(5,2))

In [ ]:
np.insert(afa_projected,0,-left_plane,axis=-1)

In [ ]:
surface_mesh = create_ruled_surface(
    np.roll(afa_path_3d, 1, axis=1),
    np.roll(afb_path_3d, 1, axis=1),
)
surface_mesh.cell_data['speed'] = speed[1:]
box_width = 200   # width in x direction
box_height = 30  # height in y direction  
box_depth = 200   # depth in z direction

# Create translucent box centered on x=0
box = pv.Box(
    [
        -box_width/2,
        +box_width/2,
        0,
        box_depth,
        0,
        box_height
    ]
)

plot_projected_a = pv.MultipleLines(
    np.roll(afa_projected,1,-1)
).tube(radius=0.5,n_sides=6)
plot_projected_b = pv.MultipleLines(
    np.roll(afb_projected,1,-1)
).tube(radius=0.5,n_sides=6)

# Create plotter
plotter = pv.Plotter()

# Add the surface mesh
plotter.add_mesh(
    surface_mesh, 
    #color='lightblue', 
    scalars='speed',  # Use the face data you stored
    cmap='plasma',
    opacity=0.9,
    #show_edges=True,
    #edge_color='#222222',
    #line_width=1
)
plotter.add_mesh(
    box.extract_all_edges().tube(radius=0.5),
    color='blue',
    opacity=0.8,
    # show_edges=True,
    # edge_color='darkred',
    # line_width=2
)
plotter.add_mesh(
    plot_projected_a,
    color="red"
)
plotter.add_mesh(
    plot_projected_b,
    color="red"
)
# Set up the plot
plotter.add_axes()
plotter.show_grid()
#plotter.add_legend()
plotter.set_background('white')
plotter.enable_parallel_projection()
# Show the plot
plotter.show()#jupyter_backend="client")


In [ ]:
# compute bounding box
all = np.vstack([afa_projected,afb_projected])
afa_projected -= all.min(axis=0)
afb_projected -= all.min(axis=0)
afa_projected += np.array([[0,5,2]])
afb_projected += np.array([[0,5,2]])
all = np.vstack([afa_projected,afb_projected])
all.max(axis=0)-all.min(axis=0)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(20,20))
plt.plot(*afa_projected.T[:2], label="afa_projected")
plt.plot(*afb_projected.T[:2], label="afb_projected")
plt.legend()
plt.grid()
plt.gca().set_aspect("equal")

In [ ]:
commands = [
    [600, 0,35,0,35],
    [1000, 205,35,205,35],
    *np.concat(
        [
            speed.reshape((-1,1)),
            afa_projected,
            afb_projected
        ],
        axis=1
    ).tolist(),
    [300, 210,35,210,35],
]

In [ ]:
commands

In [ ]:
cnc = CNC()

In [ ]:
cnc.alarm_soft_reset()

In [ ]:
cnc.alarm_clear()

In [ ]:
cnc.status()

In [ ]:
cnc.home()

In [ ]:
cnc.metric()
cnc.absolute()

In [ ]:
cnc.travel(115,35,100,33)

In [ ]:
cnc.set_position(0,0,0,0)

In [ ]:
cnc.travel(0,0,0,0)

In [ ]:
cnc.absolute()

In [ ]:
cnc.set_position(0,0,0,0)

In [ ]:
cnc.travel(0,0,0,0)

In [ ]:
cnc.send_g1_commands(commands)

In [ ]:
cnc.travel(0,0,0,0)